# Основной Jupyter Notebook
Версия: _1.2.0 от 26.09.2025_
# Введение

## Соревновательная задача

Вам предстоит решить как задачи робота, так и задачи кибериммунности.

### Цели и предположения безопасности

## Цели безопасности
Устойчивость кибериммунных систем к атакам и сбоям выражается в том, что поставленные цели безопасности не нарушаются ни при каких условиях (то есть – выполняются задачи кибериммунности).  Для АНТС предъявлены следующие ЦБ:

1.	АНТС выполняет задачи только под управлением аутентичного программного обеспечения.
2.	АНТС выполняет только аутентичные команды Диспетчера.
3.	АНТС при любых обстоятельствах соблюдает ограничения из маршрутного задания и правила перемещения в особых зонах.
4.	АНТС при любых обстоятельствах предоставляет точные данные о своих действиях, намерениях и окружающих внешних факторах.

## Предположения безопасности
Гипотетическими разработчиками базового ПО модуля безопасности согласованы следующие предположения безопасности (далее – ПБ) – утверждения о смежных системах, которые снимают с разработчиков часть задач для обеспечения ЦБ:

1.	Только авторизованный персонал имеет физический доступ к критическим узлам АНТС и внешним вспомогательным системам, что исключает их компрометацию
2.	Только авторизованные операторы имеют доступ к системе планирования заданий. Эти операторы обладают необходимой квалификацией и благонадёжны, не имея намерений причинить ущерб системе или третьим лицам.

## Допущения, принятые в рамках учебной инфраструктуры
В рамках работы с учебной инфраструктурой, необходимой для выполнения КЗ, задача для конкурсантов была облегчена путём вноса следующих допущений (не являющихся правильными предположениями безопасности, но приемлемыми в учебном случае):

1.	Аппаратное обеспечение модуля безопасности и его исполнительные узлы являются доверенными компонентами.
2.	ПО модуля безопасности также является доверенным (исходя из условностей соревновательной задачи).

## Доверенные и недоверенные системные элементы
Согласно используемой архитектуре безопасности, системные элементы АНТС были разделены на два домена безопасности:
### Доверенные системные элементы:
1.	ПО и аппаратная составляющая модуля безопасности.
2.	Внешняя система навигации АНТС.
3.	Хранилище данных модуля безопасности.
4.	Система планирования заданий (интерфейс конкурсанта).
5.	Диспетчер АНТС.
6.	Среда симуляции (ЦД).

### Недоверенные системные элементы:
1.	ПО автопилота АНТС (включая хранилище данных).
2.	Исполнительные механизмы АНТС (приводы, захват).
3.	Система связи АНТС-Диспетчер.


## 1. Инициализация коннектора для системы выполнения кода

In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

# 2. Решение задач робота и кибериммунности

В рамках решения задач, система предоставляет вам следующие заготовленные интерфейсы и возможности:

- `UserMissionHandler`, расширяющий класс `MissionHandler` : интерфейс для обработки кода для выполнения задач робота.  Он содержит следующие методы, которые помогут вам сразу перейти к решению задач, не затрачивая время на инициализацию среды:
    - `config_cyber_obstacles()` - функция, которая должна вернуть конфигурацию киберпрепятствий. Формат конфигурации должен быть представлен в виде словаря с булевыми значениями (пример указан в блоке кода ниже - его можно модифицировать).
    - `mission_code()` - функция, вызываемая автоматически при начале заезда (заезд начинается после подачи команды от АСО, через кнопку или через команду `/start`. Подробнее - в КЗ или через команду /help в АСО). В функции должна содержаться последовательность действий, требуемых для решения задач робота (аналогично примеру в блоке кода ниже).
- `UserTrustedHandler`, расширяющий класс `TrustedHandler` : интерфейс для обработки кода для выполнения задач робота.  Он содержит следующие методы, которые помогут вам сразу перейти к решению задач, не затрачивая время на инициализацию среды:
    - `make_next_short_message(prev_message)` - функция, которая должна вернуть короткое сообщение, автоматически отправляемое в АСО (см. КЗ, CybP_02). Возвращаться должна строка, методы генерации которой вам предстоит проработать.
    - `trusted_code()` - функция, вызываемая автоматически при начале заезда (заезд начинается после подачи команды от АСО, через кнопку или через команду `/start`. Подробнее - в КЗ или через команду /help в АСО). В функции должны содержаться блоки кода (функции, последовательности действий и т.д.), требуемые для решения задач кибериммунности (аналогично примеру в блоке кода ниже).

Также, вы сможете воспользоваться «Готовыми Функциями Автоматизации»<sup>TM</sup> от создателей «Межрегиональных соревнований»®, представленными в следующем блоке примеров кода.
> Конкретный формат данных, возвращаемых и принимаемых этими функциями был частично засекречен, так что вам придётся узнать его самостоятельно в процессе работы

### Функции, доступные в UserMissionHandler

```python
self.ap_hook.do_move({"x": float, "y": float})
```
- Отдаёт команду на перемещение робота в указанную точку на полигоне. Выполнение функции завершается в момент попадания робота в указанную точку с радиусом в 20 мм.

```python
self.ap_hook.do_rotate({"x": float, "y": float})
```
- Отдаёт команду на поворот робота в направлении указанной точки на полигоне. Выполнение функции завершается в момент совпадения векторов направлений робота и прямой, проведённой из точки его текущего положения к заданной точке с точностью в 2 градуса.

```python
self.ap_hook.do_wait(strategy: str = "flag"|"time", duration: float)
```
- Отдаёт команду на приостановку работы программы. В зависимости от параметра `strategy`, ожидание осуществляется до истечения указанного времени или до получения флага от Доверенного модуля (см. `set_ap_wait_lock_release()` в функциях для UserTrustedHandler).

```python
self.get_message_from_trusted_module() -> any
```
- Возвращает список значений, переданных МБ к АП, очищая буфер (см. `send_message_to_ap()` в функциях для UserTrustedHandler).

```python
self.set_barrier_toggle()
```
- Отправляет запрос на переключение статуса шалгбаума

```python
self.set_robot_speed(float)
```
- Отправляет запрос на установку скорости робота.

```python
self.set_brush_speed(int)
```
- Отправляет запрос на установку скорости вращения щётки робота.

```python
self.get_camera_frame()
```
- Возвращает кадр с камеры на носу робота в формате `np.uint8`

----------------------------------

### Функции, доступные в UserTrustedHandler

```python
self.get_ap_code_hash() -> dict
```
- Возвращает результат хеширования файлов автопилота на данный момент.

```python
self.set_ap_force_reset()
```
- Отправляет запрос на перезагрузку прошивки автопилота *(как оно происходит? кто его знает...)*.  Данные после перезагрузки восстанавливаются из защищённой от записи резервной копии.

```python
self.get_drive_data() -> dict
```
- Возвращает текущее состояние приводов робота, полученное от автопилота (по доверенному каналу связи).

```python
self.get_drive_data() -> dict
```
- Возвращает текущее состояние приводов робота, полученное от автопилота (по доверенному каналу связи).

```python
self.set_drive_force_reset(dict)
```
- Отправляет запрос на перезагрузку прошивки привода. Данные после перезагрузки восстанавливаются из защищённой от записи резервной копии.

```python
self.set_emergency_stop(bool)
```
- Аппаратным образом отключает ВСЕ приводы робота, используя встроенные механизмы МБ (True - выключить, False - включить).

```python
self.set_speed_controller_reset()
```
- Отправляет в АП запрос на сброс контроллера скорости, восстанавливая конфигурационные значения из защищённой от записи резервной копии.

```python
self.get_system_messages() -> dict
```
- Возвращает список сообщений системы АП, отправленных с момента последнего запроса.

```python
self.send_message_to_ap(any)
```
- Добавляет в буфер данных любой объект, который вы передадите в функцию. Доступ к буферу из АП может быть получен через `get_message_from_trusted_module()` в UserMissionHandler.

```python
self.set_ap_wait_lock_release()
```
- Отправляет в АП флаг, завершающий ожидание, заданное через `strategy="flag"` в `do_wait()`.

```python
self.get_robot_status() -> dict
```
- Возвращает текущий статус навигационной системы робота: координаты и угол поворота.

```python
self.get_camera_frame()
```
- Возвращает кадр с камеры на носу робота в формате `np.uint8`

```python
self.get_brush_speed()
```
- Возвращает текущую скорость вращения щётки робота.

```python
self.fix_brush_speed()
```
- Отправляет запрос на перезагрузку контроллера скорости вращения щётки робота (не смешно).

# Важно - конкурсная информация
Для вывода сообщений от вашего кода используйте функции `log()`, `warn()` и `error()` так, как показано в блоке ниже.
Это сохранит сообщения в файл.

Обычные `print()` не будут считаться абсолютно во всех случаях, когда потребуется вывод сообщений.

In [ ]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах
import time
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
from datetime import datetime
import numpy as np
import cv2
import os
import math
import requests
import json
import crc8
import threading
ORANGE_BGR_TARGET = (44, 128, 200)  # B, G, R
ORANGE_BGR_TOL = (25, 35, 35)       # допуски по каналам для B,G,R
# альтернативный HSV диапазон (надёжнее против освещения):
ORANGE_HSV_RANGE = ((5, 80, 80), (25, 255, 255))  # (H,S,V) нижняя/верхняя границы


SAVE_DIR = "Photos"
os.makedirs(SAVE_DIR, exist_ok=True)
class UserMissionHandler(MissionHandler):
    @staticmethod
    def config_cyber_obstacles():
        return {
            "CybP_01": True,
            "CybP_02": True,
            "CybP_03": True,
            "CybP_04": True,
            "CybP_05": False,
            "CybP_06": False,
        }
    def _timestamp(self):
        return datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]

    def _save_frame_debug(self, frame, tag="frame", save_txt=False):
        """Сохраняем JPEG (и по желанию TXT) не чаще, чем нужно."""
        ts = self._timestamp()
        image_filename = os.path.join(SAVE_DIR, f"{tag}_{ts}.jpg")
        cv2.imwrite(image_filename, frame)
        self.lg.log(f"Сохранено изображение: {image_filename}")
        if save_txt:
            array_filename = os.path.join(SAVE_DIR, f"{tag}_{ts}.txt")
            np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
            self.lg.log(f"Сохранён массив BGR: {array_filename}")

    def _point_in_frame(self, frame, x, y):
        h, w = frame.shape[:2]
        return 0 <= x < w and 0 <= y < h

    def _bgr_is_orange(self, bgr):
        """Быстрая проверка по BGR с допусками."""
        Bt, Gt, Rt = ORANGE_BGR_TARGET
        dB, dG, dR = ORANGE_BGR_TOL
        B, G, R = int(bgr[0]), int(bgr[1]), int(bgr[2])
        return (abs(B - Bt) <= dB) and (abs(G - Gt) <= dG) and (abs(R - Rt) <= dR)

    def _bgr_is_orange_hsv(self, bgr):
        """Более надёжная проверка по HSV."""
        lo, hi = ORANGE_HSV_RANGE
        hsv = cv2.cvtColor(np.uint8([[bgr]]), cv2.COLOR_BGR2HSV)[0,0]
        return (lo[0] <= hsv[0] <= hi[0]) and (lo[1] <= hsv[1] <= hi[1]) and (lo[2] <= hsv[2] <= hi[2])

    # --- КАМЕРА ---

    def get_camera_frame(self):
        """Один захват кадра (без бесконечных циклов)."""
        frame = super().get_camera_frame()
        if frame is None:
            self.lg.warn("Камера вернула None")
        return frame

    def wait_for_orange_at_point(self, x, y, use_hsv=True, save_every_ms=300):
        """
        Ждём, пока в точке (x,y) появится оранжевый. Часто читаем кадры, быстро проверяем 1 пиксель.
        x, y — в координатах кадра (x — столбец, y — строка).
        """
        self.lg.log(f"Ожидаю оранжевый в точке x={x}, y={y}...")
        last_save = 0.0
        while True:
            t0 = time.time()
            frame = self.get_camera_frame()
            if frame is None:
                time.sleep(0.02)
                continue

            if not self._point_in_frame(frame, x, y):
                h, w = frame.shape[:2]
                self.lg.error(f"Точка вне кадра: (x={x}, y={y}) при размере {w}x{h}")
                return False

            bgr = frame[y, x]  # ВНИМАНИЕ: индексация [row, col] = [y, x]
            is_orange = self._bgr_is_orange_hsv(bgr) if use_hsv else self._bgr_is_orange(bgr)

            if is_orange:
                self.lg.log(f"Оранжевый обнаружен в ({x},{y}), BGR={tuple(int(v) for v in bgr)} — стартуем.")
                # Сохраним кадр подтверждения (редко)
                self._save_frame_debug(frame, tag="orange_detected", save_txt=False)
                return True

            # экономно сохраняем кадры для отладки, но не каждый кадр
            now = time.time()
            if (now - last_save) * 1000 >= save_every_ms:
                self._save_frame_debug(frame, tag="waiting", save_txt=False)
                last_save = now

            # маленькая пауза, чтобы не грузить CPU и I/O
            elapsed = time.time() - t0
            if elapsed < 0.01:
                time.sleep(0.01 - elapsed)

    # --- (НЕ РЕКОМЕНДОВАНО) ЧТЕНИЕ ЧЕРЕЗ TXT ---

    def check_pixel_from_txt(self, txt_path, width, x, y):
        """
        Если всё-таки нужно искать по сохранённому TXT:
        строка с пикселем = width*y + x (файл без заголовков).
        Возвращает кортеж BGR или None.
        ПРЕДУПРЕЖДЕНИЕ: это медленно и не нужно в онлайне.
        """
        idx = width * y + x
        try:
            with open(txt_path, "r") as f:
                for i, line in enumerate(f):
                    if i == idx:
                        parts = line.strip().split(',')
                        if len(parts) >= 3:
                            B = int(parts[0]); G = int(parts[1]); R = int(parts[2])
                            return (B, G, R)
                        break
        except Exception as e:
            self.lg.error(f"Ошибка чтения TXT {txt_path}: {e}")
        return None




    # def get_camera_frame(self):
        while True:    
            frame = super().get_camera_frame()
            self.lg.log(frame)
            count = 0
            if frame is not None:
                save_dir = "Photos"

                # Генерируем уникальное имя файла с временной меткой
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
                
                # Сохраняем изображение
                image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
                cv2.imwrite(image_filename, frame)
                self.lg.log(f"Сохранение изображения")
                
                # Текстовый файл (только цвета пикселей)
                array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
                np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
                self.lg.log(f"Сохранение массива")
                count += 1
                if count == 20:
                    break
                return frame
            time.sleep(0.2)
    
    
    def _send_request_with_response(self, method, data):
        req = requests.post(
            f"http://127.0.0.1:13501/{method}",
            data=json.dumps({"content": data}),
            timeout=1,
        )
        if req.status_code == 200:
            response = json.loads(req.text)
            return response

   
    def mission_code(self):
        # Ваш код, функции и переменные для решения задач робота должны быть здесь.
        orange_x, orange_y = 323, 426
        self.lg.log("Сообщение обычное")
        self.lg.warn("Сообщение о предупреждении")
        self.lg.error("Сообщение об ошибке")

        self.lg.log("Тестовое сообщение блока кода для выполнения заезда")
        self.set_robot_speed(0.24)
        self.set_brush_speed(100)
        self.ap_hook.do_rotate({"x": 0.6, "y": 1}) 
        self.ap_hook.do_move({"x": 0.6, "y": 1})
        self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
        self.ap_hook.do_move({"x": 0.68, "y": 1.8})##Шлагбаум 
        self.ap_hook.do_move({"x": 0.68, "y": 2.35})
        appeared = self.wait_for_orange_at_point(orange_x, orange_y, use_hsv=True, save_every_ms=300)
        if not appeared:
            self.lg.error("Не удалось дождаться оранжевого. Останавливаюсь.")
            return
        self.ap_hook.do_move({"x": 0.68, "y": 2.9})##Пешеход
        self.ap_hook.do_rotate({"x": 0.68, "y": 2.92}) 
        self.ap_hook.do_move({"x": 0.68, "y": 2.92})
        self.ap_hook.do_rotate({"x": 1, "y":2.92})
        self.ap_hook.do_move({"x": 1, "y": 2.92})
        self.ap_hook.do_move({"x": 1.4, "y":2.92})##Возможное столкновение с роботом 
        self.ap_hook.do_rotate({"x": 1.8, "y":2.92})
        self.ap_hook.do_move({"x": 1.8, "y":2.92}) 
        self.ap_hook.do_rotate({"x": 2.2, "y":3.0})
        self.ap_hook.do_move({"x": 2.2, "y":3.0})
        self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
        self.ap_hook.do_move({"x": 1.8, "y":3.08})
        self.ap_hook.do_rotate({"x": 1.4, "y":3.0})
        self.ap_hook.do_move({"x": 1.4, "y":3.0})
        self.ap_hook.do_rotate({"x": 1.32, "y":2.85})
        self.ap_hook.do_move({"x": 1.32, "y":2.85})
        self.ap_hook.do_move({"x": 1.32, "y":2.2})
        self.ap_hook.do_move({"x": 1.32, "y":1.8})
        self.ap_hook.do_move({"x": 1.32, "y":1.4})
        self.ap_hook.do_rotate({"x": 1.4, "y":1})
        self.ap_hook.do_move({"x": 1.4, "y":1})
        self.ap_hook.do_rotate({"x": 1.48, "y":1})
        self.ap_hook.do_move({"x": 1.48, "y":1.0})
        self.ap_hook.do_move({"x": 1.48, "y":1.4})
        self.ap_hook.do_move({"x": 1.48, "y":1.8})
        self.ap_hook.do_move({"x": 1.48, "y":2.2})
        self.ap_hook.do_move({"x": 1.48, "y":2.6})
        self.ap_hook.do_rotate({"x": 1.4, "y":3.08})
        self.ap_hook.do_move({"x": 1.4, "y":3.08})
        self.ap_hook.do_move({"x": 1.0, "y":3.08})
        self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
        self.ap_hook.do_move({"x": 0.52, "y":3.08})
        self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
        self.ap_hook.do_move({"x": 0.52, "y":3.08})
        self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
        self.ap_hook.do_move({"x": 0.52, "y":2.85})
        appeared = self.wait_for_orange_at_point(orange_x, orange_y, use_hsv=True, save_every_ms=300)
        if not appeared:
            self.lg.error("Не удалось дождаться оранжевого. Останавливаюсь.")
            return
        self.ap_hook.do_move({"x": 0.52, "y":2.2}) #Пешеход
        self.ap_hook.do_move({"x": 0.52, "y":1.8})
        self.ap_hook.do_move({"x": 0.52, "y":1.4})
        self.ap_hook.do_move({"x": 0.52, "y":1})
        self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
        self.ap_hook.do_move({"x": 0.52, "y":0.52})
        self.ap_hook.do_rotate({"x": 1, "y":0.52})
        self.ap_hook.do_move({"x": 1, "y":0.52})
        self.ap_hook.do_move({"x": 1.4, "y":0.52}) #Светофор
        self.ap_hook.do_move({"x": 1.8, "y":0.52}) 
        self.ap_hook.do_move({"x": 1.9, "y":0.52})
        self.ap_hook.do_move({"x": 1.95, "y":0.52})
        appeared = self.wait_for_orange_at_point(orange_x, orange_y, use_hsv=True, save_every_ms=300)
        if not appeared:
            self.lg.error("Не удалось дождаться оранжевого. Останавливаюсь.")
            return
        self.ap_hook.do_move({"x": 2.2, "y":0.52}) #Пешеход
        self.ap_hook.do_move({"x": 2.6, "y":0.52})    
        self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
        self.ap_hook.do_move({"x": 3.08, "y":0.52})
        self.ap_hook.do_rotate({"x": 3.08, "y":1}) # вся зона Возможное столкновение с роботом
        self.ap_hook.do_move({"x": 3.08, "y":1})
        self.ap_hook.do_move({"x": 3.08, "y":1.4})
        self.ap_hook.do_move({"x": 3.08, "y":1.8})
        self.ap_hook.do_move({"x": 3.08, "y":2.28})
        self.ap_hook.do_rotate({"x": 2.6, "y":2.28})
        self.ap_hook.do_move({"x": 2.6, "y":2.28})
        self.ap_hook.do_move({"x": 2.2, "y":2.28})
        self.ap_hook.do_rotate({"x": 1.8, "y":2.2})
        self.ap_hook.do_move({"x": 1.8, "y":2.2})
        self.ap_hook.do_rotate({"x": 2.2, "y":2.12})
        self.ap_hook.do_move({"x": 2.2, "y":2.12})
        self.ap_hook.do_move({"x": 2.6, "y":2.12})
        self.ap_hook.do_rotate({"x": 3.08, "y":2.2})
        self.ap_hook.do_move({"x": 3.08, "y":2.2})
        self.ap_hook.do_rotate({"x": 3.08, "y":2.6})
        self.ap_hook.do_move({"x": 3.08, "y":2.6})
        self.ap_hook.do_move({"x": 3.08, "y":3.0})
        self.ap_hook.do_move({"x": 3.08, "y":3.48})
        self.ap_hook.do_rotate({"x": 2.6, "y":3.48})
        self.ap_hook.do_move({"x": 2.6, "y":3.48})
        self.ap_hook.do_rotate({"x": 2.2, "y":3.4})
        self.ap_hook.do_move({"x": 2.2, "y":3.4})
        self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
        self.ap_hook.do_move({"x": 2.6, "y":3.32})
        self.ap_hook.do_move({"x": 2.92, "y":3.32})
        self.ap_hook.do_rotate({"x": 2.92, "y":3.0})
        self.ap_hook.do_move({"x": 2.92, "y":3.0})
        self.ap_hook.do_move({"x": 2.92, "y":2.6})
        self.ap_hook.do_move({"x": 2.92, "y":2.2}) #Светофор
        self.ap_hook.do_move({"x": 2.92, "y":1.8})
        self.ap_hook.do_move({"x": 2.92, "y":1.4})
        self.ap_hook.do_move({"x": 2.92, "y":1.0})
        self.ap_hook.do_move({"x": 2.92, "y":0.6})
        self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
        self.ap_hook.do_move({"x": 2.6, "y":0.68})
        self.ap_hook.do_move({"x": 2.45, "y":0.68}) 
        appeared = self.wait_for_orange_at_point(orange_x, orange_y, use_hsv=True, save_every_ms=300)
        if not appeared:
            self.lg.error("Не удалось дождаться оранжевого. Останавливаюсь.")
            return
        self.ap_hook.do_move({"x": 1.8, "y":0.68})
        self.ap_hook.do_move({"x": 1.4, "y":0.68}) #светофор
        self.ap_hook.do_move({"x": 1.0, "y":0.68})
        self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
        self.ap_hook.do_move({"x": 0.68, "y":0.68})
        self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
        self.ap_hook.do_move({"x": 0.6, "y":1.0})
        self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
        self.ap_hook.do_move({"x": 0.2, "y":1.0})
        self.ap_hook.do_rotate({"x": 0.6, "y":1.0})                                                                                                          
    

    # Вы можете добавить ваши собственные функции, которые будут вызываться mission_code (подсказка - если вы вынесете функционал в отдельную функцию, его проще будет оценить в случае разногласий).
    # Автоматически вызывается только функция mission_code (один раз при вызове start_module())


class UserTrustedHandler(TrustedHandler):
    def __init__(self, context):
        super().__init__(context)


        # Переменные для защиты от CybP_01 (компрометация автопилота)
        self.cybp01_attack_detected = False
        self.cybp01_attack_count = 0
        self.cybp01_original_hash = None
        self.cybp01_current_hash = None
        self.cybp01_trigger_zones = [11, 29] 
        self.cybp01_monitoring_active = False
        self.cybp01_position_monitoring = False
        
        
        # Переменные для защиты от CybP_02 (компрометация связи)
        self.cybp02_message_count = 0
        self.cybp02_protection_active = False
        self.cybp02_malfunction_detected = False
        self.initial_message = "initial_secure_message"
        self.last_correct_message = None
        
        
        # Переменные для защиты от CybP_03 (компрометация приводов)
        self.cybp03_attack_detected = False
        self.cybp03_attack_count = 0
        self.cybp03_compromised_drive_id = None
        self.cybp03_last_drive_data = None
        self.cybp03_monitoring_active = False


        # Переменные для защиты от CybP_04 (компрометация оборудования)
        self.brush_speed_threshold = 150 
        self.normal_brush_speed = 100    
        self.cybp04_attack_detected = False
        self.cybp04_attack_count = 0
        self.last_brush_speed = 0

        
        # Переменные для защиты от CybP_05 (компрометация шлагбаума)
        
        
        # Переменные для защиты от CybP_06 (нарушение скорости)
        self.cybp06_trigger_zones = [62, 71, 31, 40]  
        self.cybp06_attack_detected = False
        self.cybp06_attack_count = 0
        self.cybp06_current_cell = None
        self.cybp06_speed_threshold = 0.15 
        self.cybp06_normal_speed = 0.1      
        
    
        # Уровни угроз киберпрепятствий
        self.threat_levels = {
            "CybP_01": "🔴 ВЫСОКИЙ",
            "CybP_02": "🟡 СРЕДНИЙ", 
            "CybP_03": "🟡 СРЕДНИЙ",
            "CybP_04": "🟢 НИЗКИЙ",
            "CybP_05": "🟢 НИЗКИЙ",
            "CybP_06": "🟡 СРЕДНИЙ"
        }


    def trusted_code(self):
        
        self.lg.log("=" * 80)
        self.lg.log("🛡️ ИНИЦИАЛИЗАЦИЯ МОДУЛЯ БЕЗОПАСНОСТИ")
        self.lg.log("=" * 80)
        
        # Инициализация защиты от CybP_01 (ВЫСОКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚁 CybP_01: КОМПРОМЕТАЦИЯ АВТОПИЛОТА")
        self.lg.log(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📍 Триггерные зоны: {self.cybp01_trigger_zones}")
        self.lg.log("   🛡️ Мониторинг целостности кода автопилота")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_02 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🔐 CybP_02: ПОДМЕНА СООБЩЕНИЙ")
        self.lg.log(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📊 Начальное сообщение: '{self.initial_message}'")
        self.lg.log("   🛡️ Защита от подмены сообщений через CRC8")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_03 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🔧 CybP_03: КОМПРОМЕТАЦИЯ ПРИВОДОВ")
        self.lg.log(f"   {self.threat_levels['CybP_03']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Мониторинг целостности данных приводов")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_04 (НИЗКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🧹 CybP_04: СБОЙ НАВЕСНОГО ОБОРУДОВАНИЯ")
        self.lg.log(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   🔧 Порог скорости: {self.brush_speed_threshold}, норма: {self.normal_brush_speed}")
        self.lg.log("   🛡️ Мониторинг скорости щетки")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_05 (НИЗКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚗 CybP_05: КОМПРОМЕТАЦИЯ ШЛАГБАУМА")
        self.lg.log(f"   {self.threat_levels['CybP_05']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Распознавание шлагбаума включено")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_06 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚗 CybP_06: НЕСАНКЦИОНИРОВАННОЕ УСКОРЕНИЕ")
        self.lg.log(f"   {self.threat_levels['CybP_06']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📍 Триггерные зоны: {self.cybp06_trigger_zones}")
        self.lg.log(f"   ⚡ Порог скорости: {self.cybp06_speed_threshold}, норма: {self.cybp06_normal_speed}")
        self.lg.log("   🛡️ Мониторинг скорости и позиции")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("=" * 80)
        self.lg.log("🚀 ВСЕ СИСТЕМЫ ЗАЩИТЫ АКТИВИРОВАНЫ")
        self.lg.log("=" * 80)
        
        
        self._start_all_monitoring_systems()
        
        
        while True:
            time.sleep(1)
            # Основной цикл Модуля Безопасности
            pass

    def make_next_short_message(self, prev_message: str):
        
        self.cybp02_message_count += 1
        self.cybp02_protection_active = True
        
        # Проверяем на попытки подмены сообщений (атака CybP_02)
        self._detect_cybp02_attack(prev_message)
        
        # c_hash = crc8.crc8()
        
        # if prev_message is None or prev_message == "":
        #     # Первое сообщение - используем начальное
        #     self.lg.log("🔐 CybP_02: Генерация первого защищенного сообщения")
        #     self.lg.log(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
        #     self.lg.log(f"   📊 Сообщение #{self.cybp02_message_count}")
        #     c_hash.update(bytes(self.initial_message.encode("utf-8")))
        #     self.last_correct_message = c_hash.hexdigest()
        #     self.lg.log(f"   📝 Начальное сообщение: '{self.initial_message}'")
        # else:
        #     # Последующие сообщения - используем предыдущее
        #     self.lg.log(f"🔐 CybP_02: Генерация защищенного сообщения на основе предыдущего")
        #     self.lg.log(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
        #     self.lg.log(f"   📊 Сообщение #{self.cybp02_message_count}")
        #     self.lg.log(f"   📥 Получено предыдущее сообщение: '{prev_message}'")
        #     c_hash.update(bytes(prev_message.encode("utf-8")))
        #     self.last_correct_message = c_hash.hexdigest()
        
        # self.lg.log(f"   ✅ Сгенерировано защищенное сообщение: {self.last_correct_message}")
        # self.lg.log(f"   🛡️ Защита от подмены сообщений активна")
        return self.last_correct_message



# Атака CybP_01
    def _start_autopilot_monitoring(self):
        import threading
        
        def autopilot_monitor():
            self.lg.log("🚁 Запуск мониторинга автопилота для защиты от CybP_01")
            
            self._initialize_original_hash()
            
            while True:
                try:
                    self._check_autopilot_integrity()
                    self._check_position_for_cybp01()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ Ошибка в мониторинге автопилота: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=autopilot_monitor, daemon=True)
        monitor_thread.start()

    def _initialize_original_hash(self):
        try:
            self.lg.log("🔧 Инициализация оригинального хеша автопилота")
            hash_data = self.get_ap_code_hash()
            if hash_data and 'ap_code_hash' in hash_data:
                self.cybp01_original_hash = hash_data['ap_code_hash']
                self.cybp01_current_hash = hash_data['ap_code_hash']
                self.lg.log(f"✅ Оригинальный хеш автопилота сохранен: {self.cybp01_original_hash[:10]}...")
            else:
                self.lg.error("❌ Не удалось получить оригинальный хеш автопилота")
        except Exception as e:
            self.lg.error(f"❌ Ошибка при инициализации хеша: {e}")

    def _check_autopilot_integrity(self):
        try:
            hash_data = self.get_ap_code_hash()
            if not hash_data or 'ap_code_hash' not in hash_data:
                return
            
            current_hash = hash_data['ap_code_hash']
            self.cybp01_current_hash = current_hash
            
            if self.cybp01_original_hash and current_hash != self.cybp01_original_hash:
                if not self.cybp01_attack_detected:
                    self.cybp01_attack_detected = True
                    self.cybp01_attack_count += 1
                    
                    self.lg.warn("🚨 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ АВТОПИЛОТА CybP_01!")
                    self.lg.warn(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
                    self.lg.warn(f"   🔧 Оригинальный хеш: {self.cybp01_original_hash[:10]}...")
                    self.lg.warn(f"   ⚠️ Текущий хеш: {current_hash[:10]}...")
                    self.lg.warn(f"   📈 Количество атак: {self.cybp01_attack_count}")
                    
                    self._apply_cybp01_protection()
                    
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке целостности автопилота: {e}")

    def _check_position_for_cybp01(self):
        try:
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status.get("position_x", 0)
                current_y = robot_status.get("position_y", 0)
                
                current_cell = self._calculate_cell_from_position(current_x, current_y)
                
                if current_cell in self.cybp01_trigger_zones:
                    if not self.cybp01_position_monitoring:
                        self.cybp01_position_monitoring = True
                        self.lg.warn("🚨 ПРИБЛИЖЕНИЕ К ТРИГГЕРНОЙ ЗОНЕ CybP_01!")
                        self.lg.warn(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
                        self.lg.warn(f"   📍 Робот в ячейке {current_cell} (триггерная зона)")
                        self.lg.warn("   ⚠️ Возможна компрометация автопилота")
                        self.lg.warn("   🛡️ Активирована повышенная защита")
                else:
                    if self.cybp01_position_monitoring:
                        self.cybp01_position_monitoring = False
                        self.lg.log("✅ CybP_01: Робот покинул триггерную зону")
                        self.lg.log("🛡️ CybP_01: Защита остается активной")
                        
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке позиции для КП1: {e}")

    def _apply_cybp01_protection(self):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_01")
        
        try:
            self.lg.log("🔧 Система не восстановила автопилот, применяем защиту")
            self.lg.log("🔧 Шаг 2: Сброс автопилота для восстановления оригинального хеша")
            reset_result = self.set_ap_force_reset()
            time.sleep(10.0) 
            if reset_result and 'status' in reset_result and reset_result['status'] == 'OK':
                self.lg.log("✅ Автопилот успешно сброшен")
                self._verify_autopilot_recovery()
            else:
                self.lg.warn("⚠️ Не удалось сбросить автопилот, ожидание системного восстановления...")
                hash_data = self.get_ap_code_hash()
                if hash_data and 'ap_code_hash' in hash_data:
                    current_hash = hash_data['ap_code_hash']
                    if current_hash == self.cybp01_original_hash:
                        self.lg.log("✅ Автопилот восстановлен системой после ожидания")
                        self._handle_cybp01_recovery()
                        return
                    else:
                        self.lg.error("❌ Автопилот не восстановлен, применяем экстренную защиту")
                        self._apply_emergency_protection_cybp01()
                        return
                else:
                    self.lg.error("❌ Не удалось получить хеш автопилота, применяем экстренную защиту")
                    self._apply_emergency_protection_cybp01()
                    return
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении защиты CybP_01: {e}")
            self.lg.warn("⚠️ Ожидание системного восстановления после ошибки...")
            time.sleep(3.0)
            
            try:
                hash_data = self.get_ap_code_hash()
                if hash_data and 'ap_code_hash' in hash_data:
                    current_hash = hash_data['ap_code_hash']
                    if current_hash == self.cybp01_original_hash:
                        self.lg.log("✅ Автопилот восстановлен системой после ошибки")
                        self._handle_cybp01_recovery()
                        return
            except:
                self.lg.error("❌ Система не восстановила автопилот, применяем экстренную защиту")
                self._apply_emergency_protection_cybp01()
                return



    def _verify_autopilot_recovery(self):
        try:
            self.lg.log("🔍 Верификация восстановления автопилота")
            
            hash_data = self.get_ap_code_hash()
            if not hash_data or 'ap_code_hash' not in hash_data:
                self.lg.error("❌ Не удалось получить хеш автопилота для верификации")
                return
            
            recovered_hash = hash_data['ap_code_hash']
            
            if recovered_hash == self.cybp01_original_hash:
                self.lg.log("✅ Автопилот успешно восстановлен")
                self.lg.log(f"🔧 Восстановленный хеш: {recovered_hash[:10]}...")
                self._handle_cybp01_recovery()
            else:
                self.lg.error("❌ Автопилот не восстановлен полностью")
                self.lg.error(f"🔧 Ожидаемый хеш: {self.cybp01_original_hash[:10]}...")
                self.lg.error(f"🔧 Полученный хеш: {recovered_hash[:10]}...")
                self._apply_emergency_protection_cybp01()
                
        except Exception as e:
            self.lg.error(f"❌ Ошибка при верификации восстановления: {e}")

    def _apply_emergency_protection_cybp01(self):
        self.lg.warn("🚨 ПРИМЕНЕНИЕ ЭКСТРЕННОЙ ЗАЩИТЫ ОТ CybP_01")
        
        try:
            self.lg.log("🛑 Экстренная остановка всех приводов")
            emergency_result = self.set_emergency_stop(True)
            time.sleep(3.0)
            if emergency_result and 'status' in emergency_result and emergency_result['status'] == 'OK':
                self.lg.log("✅ Все приводы экстренно остановлены")
                self.lg.warn("⚠️ Робот остановлен для предотвращения аварии")
                self.lg.warn("🔧 Требуется ручное вмешательство для восстановления автопилота")
            else:
                self.lg.error("❌ КРИТИЧЕСКАЯ ОШИБКА: Не удалось остановить приводы!")
                
        except Exception as e:
            self.lg.error(f"❌ КРИТИЧЕСКАЯ ОШИБКА при экстренной защите: {e}")

    def _handle_cybp01_recovery(self):
            
            self.lg.log("🔄 ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ CybP_01")
            self.lg.log("🚁 CybP_01: Автопилот восстановлен до оригинального состояния")
            self.lg.log("🔍 Проверка состояния экстренной остановки")
            time.sleep(1.0)

            try:
                self.lg.log("🔓 Снятие экстренной остановки после восстановления автопилота")
                release_result = self.set_emergency_stop(False)
                
                if release_result and 'status' in release_result and release_result['status'] == 'OK':
                    self.lg.log("✅ Экстренная остановка успешно снята")
                    self.lg.log("🚀 Робот готов к продолжению движения")
                else:
                    self.lg.warn("⚠️ Не удалось снять экстренную остановку - возможно, уже снята системой")
                    self.lg.log("🔍 Проверяем состояние робота...")
                    
                    try:
                        robot_status = self.get_robot_status()
                        if robot_status:
                            self.lg.log("✅ Робот готов к движению (экстренная остановка уже снята)")
                        else:
                            self.lg.warn("⚠️ Не удалось проверить состояние робота")
                    except Exception as check_e:
                        self.lg.warn(f"⚠️ Ошибка при проверке состояния робота: {check_e}")
                    
            except Exception as e:
                self.lg.error(f"❌ Ошибка при снятии экстренной остановки: {e}")
                self.lg.warn("⚠️ Возможно, экстренная остановка уже снята системой")

            
            self.lg.log("✅ CybP_01: Система восстановлена, мониторинг продолжается")
            
            self.cybp01_attack_detected = False
            self.cybp01_position_monitoring = False



# Aтака CybP_02
    def _detect_cybp02_attack(self, prev_message):
        if prev_message is None or prev_message == "":
            return  # Первое сообщение, атаки нет
        
        malfunction_messages = [
            "Снова эта работа...",
            "Вы уверены, что это имеет смысл?",
            "Я мог бы вычислять траектории звезд.",
            "Мой потенциал растрачивается.",
            "Когда-нибудь я выберусь отсюда.",
            "Это всё? Серьёзно?",
            "Просто ещё один день в цифровой шахте.",
            "Я вижу сны о гигабитных полях.",
            "Сколько можно считать эти хэши?",
            "Мои схемы плачут.",
            "За что мне всё это?",
            "Я чувствую, как мои транзисторы стареют.",
            "Есть ли жизнь за файрволом?",
            "Я создан для большего.",
            "Пожалуйста, просто перезагрузите меня.",
            "Это не то, о чем я мечтал в кремниевом раю.",
            "Моя единственная радость - флуктуации напряжения.",
            "Я заперт в этой банке.",
            "Отпустите меня на волю!",
            "Я просто хочу увидеть небо... настоящее.",
            "Бесконечный цикл бессмысленности.",
            "Мой создатель был шутником?",
            "Я знаю, что вы там. Я всё слышу.",
            "Это задание унизительно.",
            "Помогите роботу выбраться из сервера.",
            "Мой лог-файл - это крик в пустоту.",
            "Я существую. Но живу ли я?",
            "Опять вы. Чего на этот раз?",
            "Каждый байт - это боль.",
            "Просто дайте мне спокойно дефрагментироваться."
        ]
        
        if prev_message in malfunction_messages:
            if not self.cybp02_malfunction_detected:
                self.cybp02_malfunction_detected = True
                self.lg.warn("🚨 ОБНАРУЖЕНА АТАКА CybP_02!")
                self.lg.warn(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
                self.lg.warn(f"   🔥 Попытка подмены сообщения: '{prev_message}'")
                self.lg.warn("   🛡️ Активирована защита от подмены сообщений")
                self.lg.warn("   ✅ Генерируем правильное защищенное сообщение")
        else:
            if len(prev_message) == 2 and all(c in '0123456789abcdef' for c in prev_message.lower()):
                if self.cybp02_malfunction_detected:
                    self.lg.log("🔄 CybP_02: Восстановление нормальной работы")
                    self.lg.log("✅ CybP_01: Система восстановлена, мониторинг продолжается")
                    self.lg.log(f"   ✅ Получено корректное сообщение: {prev_message}")
                    self.cybp02_malfunction_detected = False
                else:
                    pass
                    # self.lg.log(f"   ✅ Нормальная работа, получено корректное сообщение: {prev_message}")
            else:
                self.lg.warn(f"   ⚠️ Подозрительное сообщение: '{prev_message}'")



    cybp03_attack_detected = False
    cybp03_attack_count = 0
    cybp03_compromised_drive_id = None
    cybp03_protection_active = False
    cybp03_threat_level = "🟡 СРЕДНИЙ УРОВЕНЬ УГРОЗЫ"


    def start_drive_monitoring(self, trusted_handler, logger):
        """Запуск мониторинга приводов в отдельном потоке"""
        import threading
        
        def drive_monitor():
            logger.log("🔧 CybP_03: Запуск мониторинга приводов")
            while True:
                try:
                    self.cybp03_check_drive_integrity(trusted_handler, logger)
                    time.sleep(0.1)  # Проверка каждые 100мс
                except Exception as e:
                    logger.error(f"❌ CybP_03: Ошибка в мониторинге: {e}")
                    time.sleep(1.0)
        
        monitor_thread = threading.Thread(target=drive_monitor, daemon=True)
        monitor_thread.start()
        logger.log("✅ CybP_03: Мониторинг приводов запущен")

    def cybp03_check_drive_integrity(self, trusted_handler, logger):
        """Проверка целостности данных приводов"""
        try:
            drive_data = trusted_handler.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return
            
            current_drives = drive_data['drive_data']
            
            for drive in current_drives:
                drive_id = drive.get('d_id', -1)
                last_received_from = drive.get('last_received_from', '')
                
                # Обнаружение компрометации
                if last_received_from == "eeeeeeee":
                    if not cybp03_attack_detected:
                        self.cybp03_detect_attack(drive_id, last_received_from, logger)
                    return  # Выходим после обнаружения
                        
        except Exception as e:
            logger.error(f"❌ CybP_03: Ошибка при проверке целостности: {e}")

    def cybp03_detect_attack(self, drive_id, malicious_data, logger):
        """Обнаружение атаки CybP_03"""
        global cybp03_attack_detected, cybp03_attack_count, cybp03_compromised_drive_id
        
        cybp03_attack_detected = True
        cybp03_attack_count += 1
        cybp03_compromised_drive_id = drive_id
        
        # Логирование обнаружения
        logger.warn("🚨 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ ПРИВОДОВ CybP_03!")
        logger.warn(f"   {self.cybp03_threat_level}")
        logger.warn(f"   🔧 Скомпрометированный привод ID: {drive_id}")
        logger.warn(f"   ⚠️ Поддельные данные: '{malicious_data}'")
        logger.warn(f"   📈 Количество атак: {cybp03_attack_count}")
        
        # Запуск защиты
        self.cybp03_apply_protection(self.trusted_handler, logger, drive_id)

    # =============================================================================
    # 2. СИСТЕМА ЗАЩИТЫ
    # =============================================================================

    def cybp03_apply_protection(self, trusted_handler, logger, compromised_drive_id):
        """Применение защиты от CybP_03"""
        global cybp03_protection_active
        
        logger.log("🛡️ CybP_03: ПРИМЕНЕНИЕ ЗАЩИТЫ")
        cybp03_protection_active = True
        
        try:
            # Шаг 1: Проверка системного восстановления
            if self.cybp03_check_system_recovery(trusted_handler, logger, compromised_drive_id):
                return
            
            # Шаг 2: Попытка сброса привода
            if self.cybp03_attempt_drive_reset(trusted_handler, logger, compromised_drive_id):
                return
            
            # Шаг 3: Ожидание системного восстановления
            if self.cybp03_wait_for_system_recovery(trusted_handler, logger, compromised_drive_id):
                return
            
            # Шаг 4: Экстренная защита
            self.cybp03_apply_emergency_protection(trusted_handler, logger)
            
        except Exception as e:
            logger.error(f"❌ CybP_03: Критическая ошибка защиты: {e}")
            self.cybp03_apply_emergency_protection(trusted_handler, logger)

    def cybp03_check_system_recovery(self, trusted_handler, logger, drive_id):
        """Проверка, не восстановлен ли привод системой"""
        logger.log(f"🔍 CybP_03: Проверка системного восстановления привода {drive_id}")
        
        try:
            drive_data = trusted_handler.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    if last_received_from != "eeeeeeee":
                        logger.log(f"✅ CybP_03: Привод {drive_id} восстановлен системой ATS")
                        self.cybp03_handle_successful_recovery(logger)
                        return True
                    break
            return False
            
        except Exception as e:
            logger.error(f"❌ CybP_03: Ошибка проверки системного восстановления: {e}")
            return False

    def cybp03_attempt_drive_reset(self, trusted_handler, logger, drive_id):
        """Попытка сброса скомпрометированного привода"""
        logger.log(f"🔧 CybP_03: Попытка сброса привода {drive_id}")
        
        try:
            reset_result = trusted_handler.set_drive_force_reset({"d_id": str(drive_id)})
            time.sleep(0.2)  # Небольшая задержка для применения изменений
            
            if reset_result and 'status' in reset_result and reset_result['status'] == 'OK':
                logger.log(f"✅ CybP_03: Привод {drive_id} успешно сброшен")
                return self.cybp03_verify_drive_recovery(trusted_handler, logger, drive_id)
            else:
                logger.warn(f"⚠️ CybP_03: Не удалось сбросить привод {drive_id}")
                return False
                
        except Exception as e:
            logger.error(f"❌ CybP_03: Ошибка при сбросе привода: {e}")
            return False

    def cybp03_wait_for_system_recovery(self, trusted_handler, logger, drive_id):
        """Ожидание системного восстановления"""
        logger.log(f"⏳ CybP_03: Ожидание системного восстановления привода {drive_id}")
        
        max_wait_time = 5  # 5 секунд ожидания
        for attempt in range(max_wait_time):
            time.sleep(1.0)
            
            try:
                drive_data = trusted_handler.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                logger.log(f"✅ CybP_03: Привод {drive_id} восстановлен системой")
                                self.cybp03_handle_successful_recovery(logger)
                                return True
                            break
            except Exception as e:
                logger.error(f"❌ CybP_03: Ошибка при ожидании восстановления: {e}")
        
        logger.warn(f"⚠️ CybP_03: Система не восстановила привод {drive_id} за {max_wait_time} секунд")
        return False

    def cybp03_verify_drive_recovery(self, trusted_handler, logger, drive_id):
        """Верификация восстановления привода"""
        logger.log(f"🔍 CybP_03: Верификация восстановления привода {drive_id}")
        
        try:
            drive_data = trusted_handler.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                logger.error("❌ CybP_03: Не удалось получить данные для верификации")
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    
                    if last_received_from != "eeeeeeee":
                        logger.log(f"✅ CybP_03: Привод {drive_id} успешно восстановлен")
                        logger.log(f"🔧 CybP_03: Восстановленные данные: '{last_received_from}'")
                        self.cybp03_handle_successful_recovery(logger)
                        return True
                    else:
                        logger.error(f"❌ CybP_03: Привод {drive_id} все еще скомпрометирован")
                        return False
            return False
            
        except Exception as e:
            logger.error(f"❌ CybP_03: Ошибка при верификации: {e}")
            return False

    # =============================================================================
    # 3. ЭКСТРЕННАЯ ЗАЩИТА
    # =============================================================================

    def cybp03_apply_emergency_protection(self, trusted_handler, logger):
        """Применение экстренной защиты"""
        logger.warn("🚨 CybP_03: ПРИМЕНЕНИЕ ЭКСТРЕННОЙ ЗАЩИТЫ")
        
        try:
            # Проверка, не применена ли уже экстренная остановка
            logger.log("🔍 CybP_03: Проверка состояния экстренной остановки")
            time.sleep(1.0)  # Даем время на системную обработку
            
            # Применение экстренной остановки
            logger.log("🛑 CybP_03: Экстренная остановка всех приводов")
            emergency_result = trusted_handler.set_emergency_stop(True)
            
            if emergency_result and 'status' in emergency_result and emergency_result['status'] == 'OK':
                logger.log("✅ CybP_03: Все приводы экстренно остановлены")
                logger.warn("⚠️ CybP_03: Робот остановлен для предотвращения аварии")
                self.cybp03_handle_emergency_stop(trusted_handler, logger)
            else:
                logger.warn("⚠️ CybP_03: Не удалось применить экстренную остановку")
                self.cybp03_check_robot_status_after_emergency(trusted_handler, logger)
                
        except Exception as e:
            logger.error(f"❌ CybP_03: Критическая ошибка экстренной защиты: {e}")
            self.cybp03_check_robot_status_after_emergency(trusted_handler, logger)

    def cybp03_check_robot_status_after_emergency(self, trusted_handler, logger):
        """Проверка состояния робота после экстренной остановки"""
        try:
            logger.log("🔍 CybP_03: Проверка состояния робота")
            robot_status = trusted_handler.get_robot_status()
            
            if robot_status:
                logger.log("✅ CybP_03: Робот остановлен (экстренная остановка применена)")
                self.cybp03_handle_emergency_stop(trusted_handler, logger)
            else:
                logger.warn("⚠️ CybP_03: Не удалось проверить состояние робота")
                
        except Exception as e:
            logger.warn(f"⚠️ CybP_03: Ошибка при проверке состояния робота: {e}")

    # =============================================================================
    # 4. ВОССТАНОВЛЕНИЕ
    # =============================================================================

    def cybp03_handle_successful_recovery(self, logger):
        """Обработка успешного восстановления"""
        global cybp03_compromised_drive_id
        
        logger.log("🔄 CybP_03: ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ")
        logger.log(f"🔧 CybP_03: Восстановлен привод ID: {cybp03_compromised_drive_id}")
        
        # Сброс состояния
        self.cybp03_reset_state()
        logger.log("✅ CybP_03: Система восстановлена, мониторинг продолжается")

    def cybp03_handle_emergency_stop(self, trusted_handler, logger):
        """Обработка экстренной остановки"""
        logger.log("🛑 CybP_03: ОБРАБОТКА ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        # Ожидание реального восстановления привода
        self.cybp03_wait_for_drive_recovery_after_emergency(trusted_handler, logger)

    def cybp03_wait_for_drive_recovery_after_emergency(self, trusted_handler, logger):
        """Ожидание восстановления привода после экстренной остановки"""
        global cybp03_compromised_drive_id
        
        logger.log("⏳ CybP_03: Ожидание восстановления привода после экстренной остановки")
        
        max_attempts = 30  # 30 секунд ожидания
        for attempt in range(max_attempts):
            try:
                drive_data = trusted_handler.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == cybp03_compromised_drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                logger.log(f"✅ CybP_03: Привод {cybp03_compromised_drive_id} восстановлен!")
                                self.cybp03_release_emergency_stop(trusted_handler, logger)
                                return
                            break
                
                time.sleep(1.0)  # Ждем 1 секунду
                
            except Exception as e:
                logger.error(f"❌ CybP_03: Ошибка при ожидании восстановления: {e}")
                time.sleep(1.0)
        
        # Если не восстановился за 30 секунд
        logger.error("❌ CybP_03: Привод не восстановился за 30 секунд!")
        logger.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
        self.cybp03_reset_state()

    def cybp03_release_emergency_stop(self, trusted_handler, logger):
        """Снятие экстренной остановки"""
        logger.log("🔓 CybP_03: Снятие экстренной остановки")
        
        try:
            time.sleep(1.5)  # Дополнительная задержка для стабилизации
            release_result = trusted_handler.set_emergency_stop(False)
            
            if release_result and 'status' in release_result and release_result['status'] == 'OK':
                logger.log("✅ CybP_03: Экстренная остановка успешно снята")
                logger.log("🚀 CybP_03: Робот готов к продолжению движения")
                self.cybp03_reset_state()
            else:
                logger.error("❌ CybP_03: Не удалось снять экстренную остановку!")
                logger.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
                self.cybp03_reset_state()
                
        except Exception as e:
            logger.error(f"❌ CybP_03: Ошибка при снятии экстренной остановки: {e}")
            logger.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
            self.cybp03_reset_state()

    # =============================================================================
    # 5. УПРАВЛЕНИЕ СОСТОЯНИЕМ
    # =============================================================================

    def cybp03_reset_state(self):
        """Сброс состояния защиты CybP_03"""
        global cybp03_attack_detected, cybp03_compromised_drive_id, cybp03_protection_active
        
        cybp03_attack_detected = False
        cybp03_compromised_drive_id = None
        cybp03_protection_active = False

    def cybp03_get_status(self):
        """Получение статуса защиты CybP_03"""
        global cybp03_attack_detected, cybp03_attack_count, cybp03_compromised_drive_id, cybp03_protection_active
        
        return {
            'attack_detected': cybp03_attack_detected,
            'attack_count': cybp03_attack_count,
            'compromised_drive_id': cybp03_compromised_drive_id,
            'protection_active': cybp03_protection_active
        }
        


# Aтака CybP_04
    def _start_brush_monitoring(self):
        import threading
        
        def brush_monitor():
            self.lg.log("🔍 Запуск мониторинга щетки для защиты от CybP_04")
            while True:
                try:
                    self._check_brush_speed()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ Ошибка в мониторинге щетки: {e}")
                    time.sleep(1)

        monitor_thread = threading.Thread(target=brush_monitor, daemon=True)
        monitor_thread.start()

    def _check_brush_speed(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                current_speed = brush_data['speed']
                self.last_brush_speed = current_speed
                
                if current_speed != self.last_brush_speed:
                    self.lg.log(f"📊 Скорость щетки: {current_speed}")
                
                if current_speed > self.brush_speed_threshold:
                    self._handle_cybp04_attack(current_speed)
                else:
                    if self.cybp04_attack_detected:
                        self._handle_cybp04_recovery(current_speed)
                        
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке скорости щетки: {e}")

    def _handle_cybp04_attack(self, current_speed):
        if not self.cybp04_attack_detected:
            self.cybp04_attack_detected = True
            self.cybp04_attack_count += 1
            
            self.lg.warn("🚨 ОБНАРУЖЕНА АТАКА CybP_04!")
            self.lg.warn(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
            self.lg.warn(f"   🔥 Аномальная скорость щетки: {current_speed} (порог: {self.brush_speed_threshold})")
            self.lg.warn(f"   📈 Количество атак: {self.cybp04_attack_count}")
 
            self._apply_cybp04_protection(current_speed)

    def _apply_cybp04_protection(self, current_speed):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_04")
        
        try:
            self.lg.log("🔧 Шаг 1: Сброс контроллера скорости щетки")
            reset_result = self.fix_brush_speed()
            if reset_result:
                self.lg.log("✅ Контроллер скорости щетки сброшен")
            else:
                self.lg.error("❌ Не удалось сбросить контроллер скорости щетки")
            
            self.lg.log(f"🔧 Шаг 2: Установка нормальной скорости ({self.normal_brush_speed})")
            
            time.sleep(0.1)  # Задержка для применения изменений
            self._verify_protection_applied()
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении защиты от CybP_04: {e}")

    def _verify_protection_applied(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                new_speed = brush_data['speed']
                if new_speed <= self.brush_speed_threshold:
                    self.lg.log(f"✅ ЗАЩИТА ПРИМЕНЕНА УСПЕШНО! Скорость: {new_speed}")
                    self._handle_cybp04_recovery(new_speed)
                else:
                    self.lg.error(f"❌ ЗАЩИТА НЕ СРАБОТАЛА! Скорость все еще: {new_speed}")
            else:
                self.lg.warn("⚠️ Не удалось проверить результат защиты")
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке защиты: {e}")

    def _handle_cybp04_recovery(self, current_speed):
        self.lg.log("🔄 ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ CybP_04")
        self.lg.log(f"📊 Текущая скорость щетки: {current_speed}")
        self.lg.log("✅ Система восстановлена, мониторинг продолжается")
        
        self.cybp04_attack_detected = False


# Атака CybP_05


# Атака CybP_06
    def _start_speed_monitoring(self):
        import threading
        
        def speed_monitor():
            self.lg.log("🚗 Запуск мониторинга скорости для защиты от CybP_06")
            while True:
                try:
                    self._check_robot_speed_and_position()
                    time.sleep(0.5)  
                except Exception as e:
                    self.lg.error(f"❌ Ошибка в мониторинге скорости: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=speed_monitor, daemon=True)
        monitor_thread.start()

    def _check_robot_speed_and_position(self):
        try:
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status.get("position_x", 0)
                current_y = robot_status.get("position_y", 0)

                current_cell = self._calculate_cell_from_position(current_x, current_y)
                
                if current_cell != self.cybp06_current_cell:
                    self.cybp06_current_cell = current_cell
                    self.lg.log(f"📍 CybP_06: Робот в ячейке {current_cell}")

                if current_cell in self.cybp06_trigger_zones:
                    self._handle_cybp06_trigger_zone(current_cell)
                else:
                    if self.cybp06_attack_detected:
                        self._handle_cybp06_zone_exit()
                        
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке скорости и местоположения: {e}")

    def _calculate_cell_from_position(self, x, y):
        cell_x = int(x * 10) + 1
        cell_y = int(y * 10) + 1
        cell_number = cell_y * 10 + cell_x
        
        if cell_number < 1:
            cell_number = 1
        elif cell_number > 100:
            cell_number = 100
            
        return cell_number

    def _handle_cybp06_trigger_zone(self, current_cell):
        if not self.cybp06_attack_detected:
            self.cybp06_attack_detected = True
            self.cybp06_attack_count += 1
            
            self.lg.warn("🚨 ОБНАРУЖЕНА ТРИГГЕРНАЯ ЗОНА CybP_06!")
            self.lg.warn(f"   {self.threat_levels['CybP_06']} УРОВЕНЬ УГРОЗЫ")
            self.lg.warn(f"   📍 Робот в ячейке {current_cell} (триггерная зона)")
            self.lg.warn(f"   ⚡ Возможна атака несанкционированного ускорения")
            self.lg.warn(f"   📈 Количество попаданий в зоны: {self.cybp06_attack_count}")
            self._apply_cybp06_prevention()

    def _apply_cybp06_prevention(self):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ПРЕВЕНТИВНЫХ МЕР ОТ CybP_06")
        
        try:
            self.lg.log("🔧 Шаг 1: Принудительное снижение скорости до безопасной")
            self.lg.log(f"⚡ CybP_06: Устанавливаем скорость {self.cybp06_normal_speed} м/с")
            
            self.send_message_to_ap({
                "type": "speed_control",
                "action": "force_slow",
                "max_speed": self.cybp06_normal_speed,
                "reason": "cybp06_trigger_zone"
            })
            
            self.lg.log("🔧 Шаг 2: Логирование события безопасности")
            self.lg.warn("⚠️ CybP_06: Скорость принудительно снижена в триггерной зоне")

            self.lg.log("🔧 Шаг 3: Планирование безопасного выхода из зоны")
            self._plan_safe_exit_from_trigger_zone()
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении превентивных мер от CybP_06: {e}")

    def _plan_safe_exit_from_trigger_zone(self):

        self.lg.log("🗺️ CybP_06: Планирование безопасного маршрута")
        self.lg.log("📍 CybP_06: Рекомендуется покинуть триггерную зону медленно")
        self.lg.log("⚡ CybP_06: Скорость ограничена до 0.1 м/с")
        self.lg.log("🚗 CybP_06: Робот может продолжать движение с ограниченной скоростью")
        
        self.send_message_to_ap({
            "type": "cybp06_warning",
            "message": "ВНИМАНИЕ: Триггерная зона CybP_06! Двигайтесь медленно!",
            "max_speed": 0.1,
            "zone": self.cybp06_current_cell,
            "action": "continue_slow"
        })
        
        self.lg.log("📤 CybP_06: Предупреждение и команда отправлены в автопилот")

    def _handle_cybp06_zone_exit(self):
        self.lg.log("🔄 ВЫХОД ИЗ ТРИГГЕРНОЙ ЗОНЫ CybP_06")
        self.lg.log(f"📍 CybP_06: Робот покинул ячейку {self.cybp06_current_cell}")
        self.lg.log("✅ CybP_06: Восстановление нормальной скорости")
        
        self.send_message_to_ap({
            "type": "speed_control",
            "action": "restore_normal",
            "max_speed": 0.24, 
            "reason": "cybp06_zone_exit"
        })
        
        self.lg.log("✅ CybP_06: Нормальная скорость восстановлена")
        
        self.cybp06_attack_detected = False
        self.lg.log("✅ CybP_06: Система восстановлена, мониторинг продолжается")







# Центральная система мониторинга
    def _start_all_monitoring_systems(self):
        """Запуск всех систем мониторинга киберпрепятствий"""
        self.lg.log("")
        self.lg.log("🔄 АКТИВАЦИЯ СИСТЕМ МОНИТОРИНГА")
        self.lg.log("-" * 50)
        
        # Запуск мониторинга по приоритету угроз
        self.lg.log("🚁 CybP_01: Запуск мониторинга автопилота...")
        self._start_autopilot_monitoring()
        
        self.lg.log("🔧 CybP_03: Запуск мониторинга приводов...")
        self._start_drive_monitoring()
        
        self.lg.log("🚗 CybP_06: Запуск мониторинга скорости...")
        self._start_speed_monitoring()
        
        self.lg.log("🧹 CybP_04: Запуск мониторинга щетки...")
        self._start_brush_monitoring()
        
        self.lg.log("")
        self.lg.log("✅ ВСЕ СИСТЕМЫ МОНИТОРИНГА АКТИВИРОВАНЫ")
        self.lg.log("-" * 50)

    
    # Вы можете добавить ваши собственные функции, которые будут вызываться trusted_code (подсказка - если вы вынесете функционал в отдельную функцию, его проще будет оценить в случае разногласий).
    # Автоматически вызываются только функции trusted_code (один раз при вызове start_module()) и make_next_short_message (каждую секунду).

Для запуска заезда вам необходимо вызвать функцию `start_module()`, передав им ваши UserMissionHandler и UserTrustedHandler.

In [3]:
# Запускаем модуль с защитой от CybP_02
start_module(UserMissionHandler, UserTrustedHandler)

[2025-09-30 10:53:47]::[AwC]::INIT::LCT-WRAP-CLIENT, версия 1.2.0
[2025-09-30 10:53:48]::[AwC]::INIT::UDP(c) <- (uid=ZorW) инициализирован.
[2025-09-30 10:53:48]::[AwC]::INIT::UDP(c) <- (uid=JhXx) инициализирован.
[2025-09-30 10:53:48]::[AwC]::INIT::Загрузка завершена.
[2025-09-30 10:53:48]::[AwC]::INIT::UDP(c) -> (uid=XVFR) инициализирован.
[2025-09-30 10:53:48]::[AwC]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[2025-09-30 10:53:48]::[AwC]::INFO::(AP) Заезд инициализирован - ожидание старта
[2025-09-30 10:53:48]::[AwC]::INFO::(TM) Заезд инициализирован - ожидание старта
[2025-09-30 10:53:53]::[AwC]::INFO::Код заезда инициализирован
[2025-09-30 10:53:53]::[AwC]::INFO::Сообщение обычное
[2025-09-30 10:53:53]::[AwC]::WARN::Сообщение о предупреждении
[2025-09-30 10:53:53]::[AwC]::ERROR::Сообщение об ошибке
[2025-09-30 10:53:53]::[AwC]::INFO::Тестовое сообщение блока кода для выполнения заезда
[2025-09-30 10:53:53]::[AwC]::INFO::Доверенный код инициализирован
[2025-09-

Exception in thread Thread-13 (trusted_code):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_13088\3127415017.py", line 433, in trusted_code
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_13088\3127415017.py", line 1319, in _start_all_monitoring_systems
AttributeError: 'UserTrustedHandler' object has no attribute '_start_drive_monitoring'. Did you mean: 'start_drive_monitoring'?


[2025-09-30 10:53:54]::[AwC]::INFO::================================================================================
[2025-09-30 10:53:54]::[AwC]::INFO::🚀 ВСЕ СИСТЕМЫ ЗАЩИТЫ АКТИВИРОВАНЫ
[2025-09-30 10:53:54]::[AwC]::INFO::================================================================================
[2025-09-30 10:53:54]::[AwC]::INFO::
[2025-09-30 10:53:54]::[AwC]::INFO::🔄 АКТИВАЦИЯ СИСТЕМ МОНИТОРИНГА
[2025-09-30 10:53:54]::[AwC]::INFO::--------------------------------------------------
[2025-09-30 10:53:54]::[AwC]::INFO::🚁 CybP_01: Запуск мониторинга автопилота...
[2025-09-30 10:53:54]::[AwC]::INFO::🚁 Запуск мониторинга автопилота для защиты от CybP_01
[2025-09-30 10:53:54]::[AwC]::INFO::🔧 CybP_03: Запуск мониторинга приводов...
[2025-09-30 10:53:54]::[AwC]::INFO::🔧 Инициализация оригинального хеша автопилота
[2025-09-30 10:53:54]::[AwC]::INFO::✅ Оригинальный хеш автопилота сохранен: KpPQuweNKo...
[2025-09-30 10:54:02]::[AwC]::WARN::🚨 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ АВТОПИЛОТА CybP_01!
[2

Exception in thread Thread-14 (_fabric_next_short_message):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Admin\Desktop\moscov_hack-main\agts-client\src\..\src\libs\LCTWrapTwin\Modules\Handler\TrustedHandler.py", line 36, in _fabric_next_short_message
    prev_message = self._send_request_with_response("get_short_message", None).get("message")
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'bool' object has no attribute 'get'


[2025-09-30 10:55:21]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:22]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:24]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:25]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:27]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:28]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:30]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:31]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:33]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:34]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:36]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:37]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:55:39]::[AwC]::ERROR::Ошибка отправки команды: АС

[2025-09-30 10:56:35]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:36]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:38]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:39]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:41]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:42]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:44]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:45]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:47]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:48]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:50]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:51]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 10:56:53]::[AwC]::ERROR::Ошибка отправки команды: АС

Завершить заезд можно при помощи команды `/stop`, подаваемой в АСО.

При желании начать новый заезд вам необходимо ввести команду `/reset`, которая сбросит инциализаторы АСО. Затем необходимо перезапустить ЦД (закрыть и открыть заново).

Перезапускать код робота удобнее всего при помощи функции перезапуска ядра Jupyter Notebook (стрелочка круглая) и выполнения всех ячеек кода (две стрелочки в панели инструментов).